

[lacentrale](https://www.lacentrale.fr/listing)



# Les voitures selon [lacentrale.fr](https://www.lacentrale.fr/listing)

<img src= ''>

L'objectif de ce projet est de collecter les données des voitures présents sur la page `https://www.lacentrale.fr/listing` du site lacentrale.fr.

Les données que nous collecterons seront les suivantes :
name
price
year
origin
registration_date
technical_inspection
first_hand
mileage
fuel_type
transmission
num_doors
num_seats
power
co2_emission
length
trunk_volume
critair_rating
combined_consumption

In [11]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [14]:

# Le module selenium.webdriver fournit toutes les implémentations de WebDriver : Firefox, Chrome, Edge, ... (sont actuellement prises en charge). 
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

# La classe Keys fournit des touches dans le clavier telles que RETURN, F1, ALT, etc.
from selenium.webdriver.common.keys import Keys
# La classe ActionsChains permet la combinaison de touches, par ex 4 fois la touche Tab 
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By

from datetime import datetime
import time
import random

import pandas as pd
# import pytest

# Lacentrale

In [15]:
from lxml import etree
def getLacentralePage(url, page = None):
    if (page != None):
        url = url + '?page=' + page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, features='lxml')

    return soup


def getDetailsCar(carCard):
    car = {
        'nom': None,
        'prix': None,
        'année': None,
        'provenance': None,
        'mise_en_circulation': None,
        'contrôle_technique': None,
        'première_main': None,
        'kilométrage': None,
        'energie': None,
        'boite_vitesse': None,
        'nb_portes': None,
        'puissance': None,
        'emission_CO2': None
    }

    dom = etree.HTML (str(carCard))


    if (carCard.find('div',class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3") != None):
        car['nom'] = carCard.find('div',class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3").text.strip()

    if (carCard.find('span',class_="PriceInformation_classifiedPrice__b-Jae") != None):
        car['prix'] = carCard.find('span',class_="PriceInformation_classifiedPrice__b-Jae").text.strip()

    if (len(dom.xpath('//*[@id="year"]/span[2]/span')) > 0):
        car['année'] = dom.xpath('//*[@id="year"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="origin"]/span[2]/span[1]')) > 0):
        car['provenance'] = dom.xpath('//*[@id="origin"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="firstCirculationDate"]/span[2]/span')) > 0):
        car['mise_en_circulation'] = dom.xpath('//*[@id="firstCirculationDate"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="technicalControl"]/span[2]/span[1]')) > 0):
        car['contrôle_technique'] = dom.xpath('//*[@id="technicalControl"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')) > 0):
        car['première_main'] = dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="mileage"]/span[2]/span[1]')) > 0):
        car['kilométrage'] = dom.xpath('//*[@id="mileage"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="energy"]/span[2]/span')) > 0):
        car['energie'] = dom.xpath('//*[@id="energy"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="gearbox"]/span[2]/span')) > 0):
        car['boite_vitesse'] = dom.xpath('//*[@id="gearbox"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="doors"]/span[2]/span')) > 0):
        car['nb_portes'] = dom.xpath('//*[@id="doors"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')) > 0):
        car['puissance'] = dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="co2"]/span[2]/div/div/div')) > 0):
        car['emission_CO2'] = dom.xpath('//*[@id="co2"]/span[2]/div/div/div')[0].text.strip()

    return car
    
def getLacentraleCars(soup):
    # Cette fonction est à lancer sur la page avec les voitures à récupérer
    # on récupère toutes les voiture
    # chaque voiture est contenu dans une div avec une classe spécifique
    
    cars = soup.find_all('div', class_=['searchCardContainer', 'boostVo__container'])
    
    data = []
    
    # Pour chaque voiture on récupère les détails qui nous intéressent
    #print(cars)
    for car in cars:
        a = 'https://www.lacentrale.fr/' + car.find('a',class_='Vehiculecard_Vehiculecard_vehiculeCard')['href']
        #print(a)
        page = getLacentralePage(a)
        data.append(getDetailsCar(page))
    
    return data



In [16]:
# L'URL DU SITE A SCRAPPER
url = "https://www.lacentrale.fr/listing"

# notre liste de voitures
cars = []

# on récupère notre page
homePage = getLacentralePage(url)
maxPage = 300

# on récupère les voitures de toutes les pages

for currentPage in range(1, maxPage+1):
    try:
        print(currentPage)
        page = getLacentralePage(url, str(currentPage))
        data = getLacentraleCars(page)
        [cars.append(car) for car in data]
    except Exception as e:
        print(e)
        pass
df = pd.DataFrame(cars)
df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,nom,prix,année,provenance,mise_en_circulation,contrôle_technique,première_main,kilométrage,energie,boite_vitesse,nb_portes,puissance,emission_CO2
0,DACIA DOKKER VAN,13 660 €,2016,France,25/10/2016,Requis,Non,82 531 km,Essence,Manuelle,5,7,D
1,LAND ROVER DISCOVERY SPORT,23 780 €,2016,France,07/02/2016,Requis,Non,220 150 km,Diesel,Automatique,5,11,E
2,AUDI A3 IV SPORTBACK,33 900 €,2021,France,14/02/2021,Non requis,Oui,79 286 km,Diesel,Automatique,5,8,C
3,PEUGEOT 508 (2E GENERATION) SW,35 640 €,2021,Importé,03/03/2021,Non requis,Non,49 416 km,Diesel,Automatique,None,10,D
4,FORD KUGA II phase 2,23 730 €,2018,France,18/04/2018,Requis,Non,161 535 km,Diesel,Manuelle,5,8,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,OPEL MOVANO II phase 2,27 150 €,2021,France,12/06/2021,Non requis,Oui,155 000 km,Diesel,Manuelle,4,8,G
5096,RENAULT ARKANA,31 230 €,2022,France,05/08/2022,Non requis,Non,22 151 km,Essence,Automatique,5,7,C
5097,FORD TRANSIT CUSTOM II phase 2,66 960 €,2024,France,30/10/2024,Non requis,Oui,1 km,Diesel,Automatique,4,7,F
5098,CUPRA LEON,35 530 €,2022,Importé,20/07/2022,Non requis,Non,40 020 km,Hybride essence électrique,Automatique,5,8,A


In [18]:
# df.info()
df.describe()

,nom,prix,année,provenance,mise_en_circulation,contrôle_technique,première_main,kilométrage,energie,boite_vitesse,nb_portes,puissance,emission_CO2
count,5100,5100,5100,5100,5100,5100,5100,5100,5100,5100,5047,5100,4830
unique,969,2162,40,2,2152,2,2,3159,8,2,4,69,7
top,PEUGEOT 208 II,16 630 €,2020,France,01/02/2020,Non requis,Non,12 km,Essence,Automatique,5,7,C
freq,120,67,881,4149,68,3672,2927,114,2645,3070,4003,998,1533
